## Bibliotecas



In [ ]:
%pip install pandas
%pip install plotly_express
%pip install missingno
%pip install scipy
%pip install nbformat
%pip install scikit-learn

## Importação


In [ ]:
import pandas as pd
import plotly_express as px
import missingno as msno
from scipy.stats import chi2_contingency
from scipy.stats import stats
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Carregamento de dados


In [ ]:
df_1 = pd.read_csv("../../../documentos/outros/base_de_dados/base_inteli_2020_2021.csv", sep = ";")
df_2 = pd.read_csv("../../../documentos/outros/base_de_dados/base_inteli_2022_2023.csv", sep = ";")
df = pd.concat([df_1, df_2])

df

## Exploração dos Dados

### Identificação das colunas numéricas e categóricas

In [ ]:
# Identifica o tipo de dado de cada coluna
df.dtypes

Considerando a descrição da coluna, a análise do tipo de dado de cada
uma delas e a definição de dados categóricos e numéricos, foi montada a seguinte tabela:

| Coluna                        | Descrição                                                         | Tipo de dado     |
|-------------------------------|-------------------------------------------------------------------|------------------|
| date                        | Data da venda do produto                                           | Numérico      |
| weekday_name                | Dia da semana em que ocorreu a venda do produto                   | Categórico       |
| sku                         | Código do produto vendido                                         | Categórico       |
| unit_price                  | Preço do produto vendido                                          | Numérico         |
| mobly_item                  | Indicação se o item é entregue e vendido pela Mobly               | Categórico       |
| shipment_type               | Modalidade de entrega do produto (se temos no estoque ou não)    | Categórico       |
| anchor_category          | Categoria macro do produto                                        | Categórico       |
| product_department       | Departamento do produto                                           | Categórico       |
| product_category         | Categoria micro do produto                                        | Categórico       |
| origin_country              | Origem do produto (nacional ou importado)                         | Categórico       |
| process_costing             | Definição se a Mobly fornece a matéria-prima para produção do produto | Categórico       |
| sku_color                   | Cor do produto                                                    | Categórico       |
| sku_height                  | Altura do produto                                                 | Numérico         |
| sku_width                   | Largura do produto                                                | Numérico         |
| sku_length                  | Comprimento do produto                                            | Numérico         |
| sku_weight                  | Peso do produto                                                   | Numérico         |
| price_status                | Status do preço daquele produto em relação à concorrência no Google Shopping | Categórico       |
| winning_price               | Preço do item loja que está ganhando no Google Shopping          | Numérico         |
| revenue                     | Receita gerada pela venda do produto                              | Numérico         |
| items_sold                  | Quantidade vendida do produto                                     | Numérico         |
| avg_website_visits_last_week| Média diária de visitas do produto no site nos últimos 7 dias   | Numérico         |
| supplier_delivery_time      | Tempo de entrega do fornecedor (em dias)                         | Numérico         |
| stock_qty                   | Quantidade em estoque                                             | Numérico         |
| flag_bundle                 | Indicação se o produto faz parte de um pacote/bundle             | Categórico       |
| revenue_bundle              | Receita gerada pela venda do pacote/bundle                        | Numérico         |
| items_sold_bundle           | Quantidade vendida do pacote/bundle                               | Numérico         |










### Estatística Descritiva


In [ ]:
# Exibe um resumo estatístico das colunas numéricas.
df.describe()

### Tipos de variáveis


#### Variáveis nominais



`sku`,`sku_color`, `sales_type`, `mobly_item`, `shipment_type`, `anchor_category`, `product_department`, `product_category`, `origin_country`, `process_costing`, `flag_bundle`

#### Variáveis Ordinais


`date`, `weekday_name`, `supplier_delivery_time`

#### Variáveis intervalares


`unit_price`, `winning_price`, `avg_website_visits_last_week`, `sku_height`, `sku_width`, `sku_lenght`, `sku_weight`, `revenue`, `items_sold`, `stock_qty`, `revenue_bundle`, `items_sold_bundle`

### Análise das correlações

Como a coluna "avg_website_visits_last_week" é exponencial é necessário aplicar o log sobre ela para então deixar a relação linear e poder fazer a análise das correlações.


In [ ]:
# Aplica o log na coluna "avg_website_visits_last_week"
df["log_avg_website_visits_last_week"] = np.log( np.array(df["avg_website_visits_last_week"] + 1) )

Feito isso, é montada a matriz de correlação, em que é possível analisar o nível de impacto entre uma variável e outra. Tal matriz é composta por valores de -1 a 1 em que:
- Quanto mais próximo de -1, mais existe uma relação inversamente proporcional entre as variáveis
- Quanto mais próximo de 1 mais existe uma relação diretamente proporcional entre as variáveis
- Quanto mais próximo de 0 menor o nível de influência entre as variáveis


In [ ]:
# Cria e exibe a matriz de correlação das colunas
correlation_matrix = df.corr(numeric_only=True)
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matriz de Correlação')
plt.show()

De acordo com a matriz é possível perceber que:
- Itens com maior relação diretamente proporcional são: revenue e items_sold, sku_lenght e sku_weight, avg_website_visits_last_week e items_sold, avg_website_visits_last_week e revenue.

- Itens com maior relação inversamente proporcional são: sku_lenght e supplier_delivery_time, unit_price e items_sold, sku_width e stock_qty


### Gráficos

#### Gráfico 1 - Quantidade de itens vendidos por macro categoria

In [ ]:
# Cria um histograma relacionando itens vendidos e macro categoria
# Configura a coluna 'anchor_category' como eixo x e a coluna 'items_sold' como eixo y
cat_macro = px.histogram(df, x="anchor_category", y="items_sold", title= "Itens vendidos por macro categoria").update_xaxes(title_text='Macro Categoria').update_yaxes(title_text='Itens Vendidos').update_xaxes(title_text='Macro Categoria').update_yaxes(title_text='Itens Vendidos')

cat_macro.show() # Exibe o histograma

**Conclusão:** As áreas com mais vendas são sofás, cadeiras de escritório e closets

#### Gráfico 2 - Quantidade de itens vendidos por micro categoria

In [ ]:
# Cria um histograma relacionando itens vendidos e micro categoria
# Configura a coluna 'product_category' como eixo x e a coluna 'items_sold' como eixo y
cat_micro = px.histogram(df, x = "product_category", y = "items_sold", title= "Itens vendidos por micro categoria").update_xaxes(title_text='Micro Categoria').update_yaxes(title_text='Itens Vendidos')

cat_micro.show() # Exibe o histograma

**Conclusão:** As áreas que acumulam mais venda são sala de estar, quarto e escritório

#### Gráfico 3- quantidade de itens vendidos por dia da semana

In [ ]:
# Cria um histograma relacionando itens vendido e dia da semana
# Configura a coluna 'weekday_name' como eixo x e a coluna 'items_sold' como eixo y
week = px.histogram(df, x="weekday_name", y="items_sold", title= "Itens vendidos por dia da semana").update_xaxes(title_text='Dia da Semana').update_yaxes(title_text='Itens Vendidos')

week.show() # Exibe o histograma

**Conclusão:** As vendas se mantém regulares durante todos os dias da semana, havendo algumas variancias não significativas


#### Gráfico 4 - Mapa de calor de itens vendidos por dia e mês

In [ ]:
# Cria uma cópia do DataFrame original para trabalhar com ele e transforma a coluna 'date' para DateTime
df_day = df
df_day['date'] = pd.to_datetime(df_day['date'], format='%Y-%m-%d')
df_day['month'] = df_day['date'].dt.month
df_day['day'] = df_day['date'].dt.day
df_day['year'] = df_day['date'].dt.year

# Cria um pivot table que calcula a média dos valores da coluna 'items_sold' com base nos valores das colunas 'month' e 'day'
df_day_heatmap = df_day.pivot_table(values='items_sold', index='month', columns='day', aggfunc='mean')

# Cria um heatmap com os valores da pivot table 'df_day_heatmap' e configura rótulos e títulos adequados
color = "deep"
heatmap_sold = px.imshow(df_day_heatmap,
                labels=dict(x="Dia", y="Mês", color="Média de itens vendidos"),
                x=df_day_heatmap.columns,
                y=df_day_heatmap.index,
                color_continuous_scale = color).update_layout(title="Mapa de Calor da Média de Itens Vendidos por Dia e Mês", xaxis_side='bottom')
heatmap_sold.show() # Exibe o heatmap

**Conclusão:** os últimos dias do mês de novembro apresentam maior número de vendas, já os últimos dias do mês de dezembro e os primeiros dias do mês de janeiro apresentam uma baixa nas vendas. Além disso, é possível perceber que durante os seis primeiros meses do meio para o fim do mês ocorre uma leve queda nas vendas também.




#### Gráfico 5 - Receita ao longo do tempo

In [ ]:
# Cria o gráfico de linha relacionando a receita ao longo do tempo
# Configura a coluna 'date' como eixo x e a coluna 'revenue' como eixo y
revenue_over_time = px.line(df, x='date', y='revenue', title='Receita ao Longo do Tempo').update_xaxes(title_text='Tempo').update_yaxes(title_text='Receita')

revenue_over_time.show() # Exibe o gráfico de linha

**Conclusão:** é possível perceber que durante o fim de novembro de todos os três anos há um grande aumento de receita, enquanto que nos meses de dezembro e janeiro existe uma tendência de queda na receita


#### Gráfico 6 - Quantidade de produtos vendidos por peso

In [ ]:
# Cria um gráfico de dispersão relacionando peso do produto com o número de vendas
# Configura a coluna 'sku_weight' como eixo x e a coluna 'items_sold' como eixo y
weight_item= px.scatter(df, x='sku_weight', y='items_sold', title='Quantidade Vendida por peso').update_xaxes(title_text='Peso do Produto').update_yaxes(title_text='Quantidade Vendida')

weight_item.show() # Exibe o gráfico de dispersão

**Conclusão:** Produtos mais leves apresentam maior quantidade de vendas


### Hipóteses

#### Gráfico 2 - Quantidade de itens vendidos por micro categoria

In [ ]:
# Cria um histograma relacionando itens vendidos e micro categoria
# Configura a coluna 'product_category' como eixo x e a coluna 'items_sold' como eixo y
cat_micro = px.histogram(df, x = "product_category", y = "items_sold", title= "Itens vendidos por micro categoria").update_xaxes(title_text='Micro Categoria').update_yaxes(title_text='Itens Vendidos')

cat_micro.show() # Exibe o histograma

Nesse gráfico, utilizamos a diferenciação por barras verticais, divididas por categoria do produto. Os gráficos de barras verticais são especialmente eficazes para comparar valores entre várias categorias ou grupos. Os valores são organizados de forma linear ao longo do eixo horizontal, tornando mais intuitiva a comparação entre as barras adjacentes.

**Premissa:** Os produtos da categoria "Sala de Estar" são os que apresentam mais vendas (800 mil).

**Hipótese:** A sala de estar é uma parte fundamental de qualquer casa ou apartamento. É o espaço onde as pessoas geralmente se reúnem para relaxar, assistir TV, conversar com a família e receber visitas. Portanto, os móveis e acessórios para a sala de estar são considerados essenciais para a maioria das pessoas, o que cria uma demanda constante por esses produtos. Além disso, é possível perceber no Gráfico 1 - Quantidade de itens vendidos por macro categoria que os sofás também são os que apresentam mais vendas, o que pode contribuir para a colocação da micro categoria Sala de Estar.

#### Gráfico 4 - Mapa de calor de itens vendidos por dia e mês

In [ ]:
# Cria um pivot table que calcula a média dos valores da coluna 'items_sold' com base nos valores das colunas 'month' e 'day'
df_day_heatmap = df_day.pivot_table(values='items_sold', index='month', columns='day', aggfunc='mean')

# Cria um heatmap com os valores da pivot table 'df_day_heatmap' e configura rótulos e títulos adequados
color = "deep"
heatmap_sold = px.imshow(df_day_heatmap,
                labels=dict(x="Dia", y="Mês", color="Média de itens vendidos"),
                x=df_day_heatmap.columns,
                y=df_day_heatmap.index,
                color_continuous_scale = color).update_layout(title="Mapa de Calor da Média de Itens Vendidos por Dia e Mês", xaxis_side='bottom')
heatmap_sold.show() # Exibe o heatmap



Para esse gráfico, pensamos em um mapa de calor por deixar evidente e destacado em cores os períodos em que houve uma maior incidência de vendas, facilitando a visualização. Cada coluna representa um dia e cada linha representa um mês.

**Premissa:** Com base na análise do gráfico de calor, é evidente que as vendas da Mobly mantêm uma certa constância ao longo do ano, com exceção de um aumento significativo no final do mês de novembro.

**Hipótese:** O aumento acentuado nas vendas no final de novembro, conforme observado no gráfico de calor, pode ser atribuído à participação da Mobly na Black Friday. A Black Friday é um evento de compras amplamente reconhecido, caracterizado por descontos significativos e promoções especiais oferecidas por várias empresas.

Essa hipótese sugere que a Mobly aproveita a Black Friday como uma oportunidade estratégica para impulsionar suas vendas. Durante esse período, a empresa pode adotar uma estratégia agressiva de marketing, oferecendo descontos substanciais em seus produtos ou promovendo ofertas exclusivas para atrair um grande volume de compradores. Esse aumento nas atividades promocionais e nas ofertas especiais leva a um aumento nas vendas, que é claramente refletido no gráfico de calor.

A constância nas vendas ao longo do ano, com exceção desse pico em novembro, indica que a Mobly provavelmente planeja cuidadosamente suas operações e estratégias de vendas para manter um equilíbrio entre a estabilidade de receita durante o ano e a maximização das vendas durante eventos como a Black Friday. Portanto, a hipótese enfatiza a importância estratégica da Black Friday como um período de aumento significativo nas vendas, graças às suas ofertas especiais e promoções. Um indício da veracidade dessa suposição, é que é um evento sazonal, ou seja, é perceptível em todos os quatro anos da base de dados (2020-2023).

#### Gráfico 6 - Quantidade vendida por peso

In [ ]:
# Cria um gráfico de dispersão relacionando peso do produto com o número de vendas
# Configura a coluna 'sku_weight' como eixo x e a coluna 'items_sold' como eixo y
weight_item= px.scatter(df_day, x='sku_weight', y='items_sold', title='Quantidade Vendida por peso').update_xaxes(title_text='Peso do Produto').update_yaxes(title_text='Quantidade Vendida')

weight_item.show() # Exibe o gráfico de dispersão

Neste gráfico tivemos o objetivo de transmitir de forma visual a relação do peso dos produtos, com seu volume de vendas. Optamos por um gráfico de dispersão (Scatter Plot), que utiliza pontos para apresentar as vendas

**Premissa:** Produtos mais leves são mais vendidos.

**Hipótese:** A relação entre o peso de um produto e suas vendas é inversamente proporcional, indicando que produtos mais leves tendem a ser mais vendidos do que produtos mais pesados, consumidores geralmente preferem produtos que sejam mais fáceis de manusear, transportar e armazenar. Produtos mais leves podem oferecer maior conveniência e praticidade.Além disso, estes produtos muitas vezes são acompanhados de preços, e custo de entrega mais baixos. Por exemplo, o custo de entrega de uma luminária geralmente é mais baixo que o custo de entrega de um armário. Além disso, a instalação de um produto leve costuma ser muito mais simples e prática do que a de um produto mais pesado.

Ressalta-se que na apresentação de validação com os parceiros ao final da sprint 2, foi pontuado que essa relação entre menores pesos apresentarem mais vendas não é causal, ou seja, não são os menores pesos que influenciam diretamente na hora da compra. Foi pontuado que isso acontece por, geralmente, os consumidores optarem por levar mais unidades de produtos mais baratos e, também, mais leves, como por exemplo um kit de cadeiras apresenta mais saídas de um produto em uma única compra do que a compra de uma unidade de sofá.

### Tratamento de _values missing_

Por meio do código abaixo é possível ter ciência de quais colunas possuem dados nulos:

In [ ]:
# Mostra quais tabelas possuem valores nulos
df_day.isnull().sum()

A partir dessa identificação, nós optamos por remover as linhas nulas das tabelas: 'sku_height', 'sku_width', 'sku_length' e 'sku_weight'.

In [ ]:
## Lista de colunas de onde retiraremos as linhas nulas.
columns_to_verify = ['sku_height', 'sku_width', 'sku_length', 'sku_weight']

## Remove as linhas nulas apenas nas colunas especificadas
df_filtred = df_day.dropna(subset=columns_to_verify)

In [ ]:
## Verifica se os dados nulos foram removidos.
df_filtred.isnull().sum()

### Limpeza de Dados

#### Remoção da coluna mobly_item

Optamos por excluir a coluna "mobly_item" devido a sua insignificância para o modelo, uma vez que não há alguma informação que possa ser aproveitada pelo modelo preditivo. Sendo assim, o histograma abaixo consegue evidenciar tal situação:

In [ ]:
# Hitograma que evidencia as informações da coluna 'mobly_item'
plt.hist(df_day['mobly_item'], bins='auto')
plt.show()

Sendo assim, nós optamos por excluir tal coluna:

In [ ]:
# Exclui a coluna "mobly_item"
df_no_mobly_item = df_filtred.drop(["mobly_item"], axis=1)

#### Remoção das colunas flag_bundle, revenue_bundle, items_sold_bundle

Optamos por excluir as colunas `flag_bundle`, `revenue_bundle`, `items_sold_bundle` devido a sua complexidade para o modelo, uma vez que os podutos presentes na base de dados são cadastrados como individuais, algo mais simples para o aprendizado do modelo. Sendo assim, abaixo será realizada a exclusão dessas colunas:

In [ ]:
# Exclui as colunas "flag_bundle", "revenue_bundle", "items_sold_bundle"
df_no_bundles = df_no_mobly_item.drop(["flag_bundle", "revenue_bundle", "items_sold_bundle"], axis=1)

In [ ]:
# Verifica o novo dataframe
df_no_bundles

#### Remoção da coluna supplier_delivery_time

Optamos por excluir a coluna "supplier_delivery_time" devido a validação realizada no final da sprint 2, uma vez que o parceiro apontou que para a aprendizagem do modelo, tal coluna não terá tantas informações necessárias, dado que poucos fornecedores tem um prazo menor do que quatro dias. Sendo assim, abaixo há o código que realiza tal deleção:

In [ ]:
# Exclui a coluna "supplier_delivery_time"
df_no_supplier_delivery_time = df_no_bundles.drop(["supplier_delivery_time"], axis=1)

In [ ]:
# Verifica o novo dataframe
df_no_supplier_delivery_time

#### Remoção das colunas sku_length, sku_height, sku_width e sku_weight

Optamos por realizar a exclusão de tais colunas devido a sua insignificância para o modelo. Sendo assim, abaixo há o código que realiza a eliminação de tais colunas:

In [ ]:
# Exclui as colunas "sku_length", "sku_height", "sku_width", "sku_weight"
df_no_characteristics = df_no_supplier_delivery_time.drop(["sku_length", "sku_height", "sku_width", "sku_weight"], axis=1)

# Verificando o novo dataframe

In [ ]:
# Exibe o dataframe
df_no_characteristics

In [ ]:
# Baixa o dataframe na pasta do notebook atual
df_no_characteristics.to_csv('mobly_data_no_codification.csv', index=False)

### Outlier e Normalização das variáveis numéricas

O grupo optou por dois métodos de identificação de outliers:

> O método do z-score foi escolhido pelo grupo devido a sua assertividade com dados que possuem um comportamento de distribuição normal ou aproximadamente normal, uma vez que iremos normalizar todos os dados possíveis.



> Já o método de bloxpot foi escolhido devido a sua visualização mais clara dos outliers, além de que ele pode ter uma assertividade um pouco maior do que o método do z-score em determinados dados.

Assim, esses dois métodos serão utilizados para que possamos entender melhor os outliers, além de que fornece uma possibilidade de comparação entre os outliers identificados.



#### Verificando outlier e normalizando dados de unit_price

In [ ]:
# Verifica a quantidade de registros no dataframe
df_no_characteristics.shape

Antes de identificar o outlier da coluna "unit_price" é necessário verificar se os dados possui uma distribuição normal ou assimétrica. Dessa forma, tal análise está sendo realizada por meio do histograma abaixo:

In [ ]:
# Exibe um histograma que evidencia o comportamento dos dados
plt.hist(df_no_characteristics['unit_price'], bins='auto')
plt.show()

Por meio do gráfico acima é possível concluir que os dados da coluna unit_price possui o comportamento exponencial. Dessa forma, é necessário normalizar os dados para que os dados assuma um comportamento linear:

In [ ]:
# Cria uma nova coluna apenas com os dados de 'unit_price' normalizados a partir de log
df_no_characteristics['log_unit_price'] = np.log(df_no_characteristics['unit_price'] + 1)
df_no_characteristics['log_unit_price']

In [ ]:
# Cria um histograma que evidencia a normalização dos dados de 'unit_price'
plt.hist(df_no_characteristics['log_unit_price'], bins='auto')
plt.show()

Em comparação com o gráfico anterior, os dados da coluna "unit_price" foram normalizados, apresentando uma distribuição próxima da normalidade. Sendo assim, é possível identificar os outliers dessa coluna por meio do método do z-score e do boxplot:



In [ ]:
# Armazena o z-score de cada valor da coluna
z_scores = np.abs(stats.zscore(df_no_characteristics['log_unit_price']))
print(z_scores)

# Cria um novo dataframe somente com os dados correspondentes ao z-score
df_outliers = df_no_characteristics[(z_scores > 3)]
df_outliers['log_unit_price']

In [ ]:
# Exibe o Bloxpot que evidencia os outliers da coluna de 'unit_price' com dados normalizados
sns.boxplot(x=df_no_characteristics['log_unit_price'])

O gráfico acima e o método z-score evidenciam diversos outliers relacionados ao preço unitário dos produtos da Mobly. Porém, como se trata de preço unitário, o tratamento dessas anomalias precisam ser realizados com cautela, dado que tal informação é muito relevante para o modelo preditivo. Sendo assim, em um primeiro momento, optamos por não tratar esses outliers devido a sua importância.

#### Verificando outliers e normalizando dados de revenue


Antes de identificar o outlier da coluna "revenue" é necessário verificar se os dados possui uma distribuição normal ou assimétrica. Dessa forma, tal análise está sendo realizada por meio do histograma abaixo:

In [ ]:
# Cria e exibe um histograma que evidencia o comportamento dos dados da coluna 'revenue'
revenue_distribution_histogram = px.histogram(df_no_characteristics, df_no_characteristics['revenue'], nbins = 50)
revenue_distribution_histogram.show()

Por meio do gráfico acima é possível concluir que os dados da coluna "revenue" possui uma distribuição assimétrica. Dessa forma, é necessário normalizar os dados para que os dados assuma uma distribuição normal ou próxima da normalidade:

In [ ]:
# Cria uma nova coluna apenas com os dados de 'revenue' normalizados a partir de log
df_no_characteristics['log_revenue'] = np.log(df_no_characteristics['revenue'] + 1)
df_no_characteristics['log_revenue']

In [ ]:
# Cria e exibe um histograma que evidencia a normalização dos dados de 'revenue'
plt.hist(df_no_characteristics['log_revenue'], bins='auto')
plt.show()

Em comparação com o gráfico anterior, os dados da coluna "revenue" apresentam uma distribuição próxima da normalidade. Sendo assim, é possível identificar os outliers dessa coluna por meio do método do z-score e do boxplot:



In [ ]:
# Cria a coluna "log_revenue"
log_revenue = df_no_characteristics['log_revenue']

# Calcula a média dos dados
mean = log_revenue.mean()

# Calcula o desvio-padrão
standard_deviation = log_revenue.std()

# Calcula o z-score
z_scores = (log_revenue - mean) / standard_deviation
z_scores = np.abs(z_scores)

# Cria um novo dataframe somente com os dados correspondentes ao z-score
df_outliers = df_no_characteristics[(z_scores > 3)]
df_outliers["log_revenue"]



> Foi utilizado o método acima para calcular os outliers da coluna "revenue" porque o método que estava sendo utilizado antes, o qual está evidenciado logo abaixo, estava retornando "NaN" para os valores da coluna normalizados:



In [ ]:
# Armazena o z-score de cada valor da coluna
z_scores = np.abs(stats.zscore(df_no_characteristics['log_revenue']))

print(z_scores)

# Cria um novo dataframe somente com os dados correspondentes ao z-score
df_outliers = df_no_characteristics[(z_scores > 3)]
df_outliers['log_revenue']

In [ ]:
# Exibe um Bloxpot que evidencia os outliers da coluna de 'revenue' com dados normalizados
sns.boxplot(x=df_no_characteristics['log_revenue'])

O bloxpot acima evidencia que a coluna "log_revenue" não possui outliers, fazendo com que não seja necessário, em um primeiro momento, o tratamento de outliers de tal coluna.

#### Verificando outliers com os dados já normalizados de avg_website_visits_last_week

Antes de identificar o outlier da coluna "avg_website_visits_last_week" é necessário verificar se os dados possui uma distribuição normal ou assimétrica. Dessa forma, tal análise está sendo realizada por meio do histograma abaixo:

In [ ]:
# Cria e exibe um histograma que evidencia o comportamento dos dados da coluna 'avg_website_visits_last_week'
plt.hist(df_no_characteristics['avg_website_visits_last_week'], bins='auto')
plt.show()

Por meio do gráfico acima é possível concluir que os dados da coluna "avg_website_visits_last_week" possui uma distribuição assimétrica. Dessa forma, no tópico de "Análise das Correlações", os dados dessa coluna já foram normalizados, algo que pode ser verificado no gráfico abaixo:

In [ ]:
# Cria e exibe um histograma que evidencia a normalização dos dados de 'avg_website_visits_last_week'
plt.hist(df_no_characteristics['log_avg_website_visits_last_week'], bins='auto')
plt.show()

Em comparação com o gráfico anterior, os dados da coluna "avg_website_visits_last_week" foram normalizados, apresentando uma distribuição próxima da normalidade. Sendo assim, por meio desse comportamento, é possível identificar os outliers dessa coluna por meio do método do z-score e do boxplot:



In [ ]:
# Armazena o z-score de cada valor da coluna
z_scores = np.abs(stats.zscore(df_no_characteristics['log_avg_website_visits_last_week']))

print(z_scores)

# Cria um novo dataframe somente com os dados correspondentes ao z-score
df_outliers = df_no_characteristics[(z_scores < -3) | (z_scores > 3)]
df_outliers['log_avg_website_visits_last_week']

In [ ]:
# Exibe um Bloxpot que evidencia os outliers da coluna de 'avg_website_visits_last_week' com dados normalizados
sns.boxplot(x=df_no_characteristics['log_avg_website_visits_last_week'])

Tanto o gráfico acima quanto o método do z-score evidenciam que há diversos outliers. Assim, optamos por entender melhor o comportamento dos outliers à esquerda, os quais foram evidenciados pelo bloxpot:

In [ ]:
# Verifica características dos produtos que contém nenhuma visita nos últimos 7 dias
df_no_bundles[(df_no_characteristics['log_avg_website_visits_last_week'] == 0) & (df_no_characteristics['items_sold'] == 0)].head(100)

#### Verificando outliers de stock_qty

Antes de identificar o outlier da coluna "stock_qty" é necessário verificar se os dados possui uma distribuição normal ou assimétrica. Dessa forma, tal análise está sendo realizada por meio do histograma abaixo:

In [ ]:
# Cria e exibe um histograma que evidencia o comportamento dos dados da coluna 'stock_qty'
stock_distribution_histogram = px.histogram(df_no_characteristics, df_no_characteristics['stock_qty'], nbins = 50)
stock_distribution_histogram.show()

Por meio do gráfico acima é possível concluir que os dados da coluna "stock_qty" possui uma distribuição assimétrica. Dessa forma, é necessário normalizar os dados para que os dados assuma uma distribuição normal:

In [ ]:
# Cria uma nova coluna apenas com os dados de 'stock_qty' normalizados a partir de log
df_no_characteristics['log_stock'] = np.log(df_no_characteristics['stock_qty'] + 1)
df_no_characteristics['log_stock']

In [ ]:
# Cria e exibe um histograma que evidencia a normalização dos dados de 'stock_qty'
distribution_histogram_log_stock = px.histogram(df_no_characteristics, df_no_characteristics['log_stock'], nbins = 50)
distribution_histogram_log_stock.show()

Em comparação com o gráfico anterior, os dados da coluna "stock_qty" foram normalizados, apresentando uma distribuição próxima da normalidade. Sendo assim, por meio desse comportamento, é possível identificar os outliers dessa coluna por meio do método do z-score:


In [ ]:
# Armazena o z-score de cada valor da coluna
z_scores = np.abs(stats.zscore(df_no_characteristics['log_stock']))

# Cria um novo dataframe somente com os dados correspondentes ao z-score
df_outliers = df_no_characteristics[(z_scores > 3)]
df_outliers['log_stock']

In [ ]:
# Exibe um Bloxpot que evidencia os outliers da coluna de 'stock_qty' com dados normalizados
sns.boxplot(x=df_no_characteristics['log_stock'])

Tanto o bloxpot quanto o método do z-score evidenciam que não há outliers na coluna "log_stock", fazendo com que não seja necessário o tratamento desses outliers.

#### Verificando outliers e normalizando dados de items_sold

Antes de identificar o outlier da coluna "items_sold" é necessário verificar se os dados possui uma distribuição normal ou assimétrica. Dessa forma, tal análise está sendo realizada por meio do histograma abaixo:

In [ ]:
# Cria e exibe um histograma que evidencia o comportamento dos dados da coluna 'items_sold'
items_sold_distribution_histogram = px.histogram(df_no_characteristics, df_no_characteristics['items_sold'], nbins = 50)
items_sold_distribution_histogram.show()

Por meio do gráfico acima é possível concluir que os dados da coluna "items_sold" possui uma distribuição assimétrica. Dessa forma, é necessário normalizar os dados para que os dados assuma uma distribuição normal:

In [ ]:
# Cria uma nova coluna apenas com os dados de 'items_sold' normalizados a partir de log
df_no_characteristics['log_items_sold'] = np.log(df_no_characteristics['items_sold'] + 1)
df_no_characteristics['log_items_sold']

In [ ]:
# Cria e exibe um histograma que evidencia a normalização dos dados de 'items_sold'
distribution_histogram_log_items_sold = px.histogram(df_no_characteristics, np.log(df_no_characteristics['items_sold'] + 1), nbins = 50)
distribution_histogram_log_items_sold.show()

Em comparação com o gráfico anterior, os dados da coluna "items_sold" foram normalizados, apresentando uma distribuição próxima da normalidade. Sendo assim, por meio desse comportamento, é possível identificar os outliers dessa coluna por meio do método do z-score e do boxplot:


In [ ]:
# Armazena o z-score de cada valor da coluna
z_scores = np.abs(stats.zscore(df_no_characteristics['log_items_sold']))
print(z_scores)

# Cria um novo dataframe somente com os dados correspondentes ao z-score
df_outliers = df_no_characteristics[(z_scores > 3)]
df_outliers['log_items_sold']

In [ ]:
# Exibe um Bloxpot que evidencia os outliers da coluna de 'items_sold' com dados normalizados
sns.boxplot(x=df_no_characteristics['log_items_sold'])

O gráfico acima e o método do z-score evidenciam que há diversos outliers relacionados à quantidade de produtos vendidos da Mobly. Porém, o boxplot evidencia há um outiler mais destoante entre os demais, o qual é maior que 6. Dessa forma, buscamos entender um pouco melhor sobre as características desses produtos que tiveram mais itens vendidos:

In [ ]:
# Acessa os produtos que correspondem aos outliers mais destoantes
df_no_characteristics[(df_no_characteristics['log_items_sold'] > 6)]

Portanto, por meio do entendimento dessas características, optamos por não tratarmos esses outliers nesse primeiro momento, haja vista que são produtos que possuem uma alta quantidade vendida, um preço mais acessível e o seu pico de venda é durante o período da blackfriday, conforme demonstra a tabela acima. Sendo assim, o modelo conseguirá entender com exatidão a quantidade de itens vendidos de tais produtos, fazendo com que o estoque esteja sempre abastecido corretamente.

#### Normalização de dados das colunas relacionadas às características dos produtos:
* sku_height
* sku_width
* sku_length
* sku_weight


Como é possível observar na tabela acima, os dados das colunas relacionadas às características dos produtos possuem uma normalização, ou seja, estão na mesma escala.

### Codificação das variáveis categóricas

In [ ]:
# Captura as variáveis categóricas
categorical_variables = ['weekday_name', 'sku', 'shipment_type', 'origin_country', 'process_costing', 'sku_color', 'price_status', 'product_category']
categorical_variables_df = df_no_characteristics[categorical_variables]
categorical_variables_df.head(5)

Como há diversos tipos de variáveis categóricas, será utilizada duas formas de codificar tais variáveis:
* 1°: Utilizando o `LabelEncoder` do `sklearn`
* 2°: Utilizando o `map` do `Pandas`

In [ ]:
# ImportaLabelEncoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

#### Codificação da variável categórica ordinal "weekday_name"

A "weekday_name" possui diferentes dados, uma vez que possui os nomes dos dias da semana. Dessa forma, será utilizado o `LabelEncoder` para codificar as variáveis:

In [ ]:
# Codifica a coluna 'weekday_name' utilizando o LabelEncoder
df_no_characteristics['weekday_name'] = le.fit_transform(df_no_characteristics['weekday_name'])
df_no_characteristics.head(5)

#### Codificação da variável categórica nominal "shipment_type"

A "shipment_type" possui dois tipos de dados: `crossdocking` e `próprio`. Sendo assim, é possível utilizar a função `map` do Pandas para codificar as variáveis:

In [ ]:
# Codifica a coluna 'shipment_type' utilizando map
df_no_characteristics['shipment_type'] = df_no_characteristics['shipment_type'].map({'crossdocking': 0, 'próprio': 1})

In [ ]:
# Verifica a codificação
df_no_characteristics['shipment_type']

#### Codificação da variável categórica nominal "origin_country"


A "origin_country" possui dois tipos de dados: `Nacional` e `Importado`. Sendo assim, é possível utilizar a função `map` do Pandas para codificar as variáveis:

In [ ]:
# Codifica a coluna 'origin_country' utilizando map
df_no_characteristics['origin_country'] = df_no_characteristics['origin_country'].map({'Nacional': 0, 'Importado': 1})

In [ ]:
# Verifica a codificação
df_no_characteristics['origin_country']

#### Codificação da variável categórica nominal "process_costing"


A "process_costing" possui dois tipos de dados: `yes` e `no`. Sendo assim, é possível utilizar a função `map` do Pandas para codificar as variáveis:

In [ ]:
# Codifica a coluna 'process_costing' utilizando map
df_no_characteristics['process_costing'] = df_no_characteristics['process_costing'].map({'yes': 0, 'no': 1})

In [ ]:
# Verifica a codificação
df_no_characteristics['process_costing']

#### Codificação da variável categórica nominal "sku_color"


A "sku_color" possui diferentes dados, uma vez que possui os nomes de várias cores. Dessa forma, será utilizado o `LabelEncoder` para codificar as variáveis:

In [ ]:
# Codifica a coluna 'sku_color' utilizando o LabelEncoder
df_no_characteristics['sku_color'] = le.fit_transform(df_no_characteristics['sku_color'])
df_no_characteristics.head(5)

#### Codificação da variável categórica nominal "anchor_category"


A coluna "anchor_category" possui diferentes dados, uma vez que possui os nomes de várias cores. Dessa forma, será utilizado o `LabelEncoder` para codificar as variáveis:

In [ ]:
# Codifica a coluna 'anchor_category' utilizando o LabelEncoder
df_no_characteristics['anchor_category'] = le.fit_transform(df_no_characteristics['anchor_category'])
df_no_characteristics.head(5)

#### Codificação da variável categórica nominal "product_category"


A coluna "product_category" possui diferentes dados, uma vez que possui os nomes de várias cores. Dessa forma, será utilizado o `LabelEncoder` para codificar as variáveis:

In [ ]:
# Codifica a coluna 'product_category' utilizando o LabelEncoder
df_no_characteristics['product_category'] = le.fit_transform(df_no_characteristics['product_category'])
df_no_characteristics.head(5)

#### Codificação da variável categórica nominal "sku"


A coluna "sku" possui diferentes dados, uma vez que possui os nomes de várias cores. Dessa forma, será utilizado o `LabelEncoder` para codificar as variáveis:

In [ ]:
# Codifica a coluna 'sku' utilizando o LabelEncoder
df_no_characteristics['sku'] = le.fit_transform(df_no_characteristics['sku'])
df_no_characteristics.head(5)

#### Codificação da variável categórica nominal "product_department"

A coluna "product_department" possui diferentes dados, uma vez que possui os nomes de várias cores. Dessa forma, será utilizado o `LabelEncoder` para codificar as variáveis:

In [ ]:
# Codifica a coluna 'product_department' utilizando o LabelEncoder
df_no_characteristics['product_department'] = le.fit_transform(df_no_characteristics['product_department'])
df_no_characteristics.head(5)

#### Codificação da variável categórica nominal "price_staus"

A coluna "price_status" possui diferentes dados, uma vez que possui os nomes de várias cores. Dessa forma, será utilizado o `LabelEncoder` para codificar as variáveis:

In [ ]:
# Codifica a coluna 'product_department' utilizando o LabelEncoder
df_no_characteristics['price_status'] = le.fit_transform(df_no_characteristics['price_status'])
df_no_characteristics.head(5)

In [ ]:
# Exibe o dataframe
df_no_characteristics

# Baixando o dataset 

In [ ]:
# Baixa o dataframe na pasta do notebook atual
df_no_characteristics.to_csv('mobly_data.csv', index=False)